In [1]:
import cv2 as cv
import numpy as np
import pyttsx3
from sklearn.cluster import KMeans
from collections import Counter

 
cap = cv.VideoCapture(r"C:\Users\admin\Downloads\Top 10 Safety Vest For Construction For Men And Women.mp4")
whT = 320
confThreshold =0.5
nmsThreshold= 0.2
 
#### LOAD MODEL
## Coco Names
classesFile = r"C:\Users\admin\Downloads\wobot\obj.names"
classNames = []
with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
print(classNames)
## Model Files
modelConfiguration = r"C:\Users\admin\Downloads\wobot\yolov4-tiny-custom.cfg"
modelWeights = r"C:\Users\admin\Downloads\wobot\yolov4-tiny-custom_best.weights"
net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)


clt = KMeans(n_clusters=3)
def palette_perc(k_cluster):
    width = 300
    b=k_cluster.cluster_centers_[0]
    return b

def findObjects(outputs,img):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w,h = int(det[2]*wT) , int(det[3]*hT)
                x,y = int((det[0]*wT)-w/2) , int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))
    
    indices = cv.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)
 
    for i in indices:
        i = i[0]
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        cv.putText(img,f'{classNames[classIds[i]].upper()} {int(confs[i]*100)}%',
                  (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
        a=classNames[classIds[i]].upper()
        
        if a=="HELMET":
            im=img[y:y+h,x:x+w]
            i=im.reshape(-1,3)
            if i.shape!=(0,3):
                clt_1 = clt.fit(im.reshape(-1, 3))
                b=palette_perc(clt_1)
                cv.rectangle(img, (x, y), (x+w,y+h), (b[0],b[1],b[2]), 2)
        else:
            cv.rectangle(img, (x, y), (x+w,y+h), (255, 0, 255), 2)


            
# frame_width = int(cap.get(3))
# frame_height = int(cap.get(4))  
# size = (frame_width, frame_height)
# result = cv.VideoWriter('wobot.avi', 
#                          cv.VideoWriter_fourcc(*'MJPG'),
#                          10, size)            
            
            
while True:
    success, img = cap.read()
    blob = cv.dnn.blobFromImage(img, 1 / 255, (whT, whT), [0, 0, 0], 1, crop=False)
    net.setInput(blob)
    layersNames = net.getLayerNames()
    outputNames = [(layersNames[i[0] - 1]) for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(outputNames)
    findObjects(outputs,img)
    cv.imshow('Image', img)
    #result.write(img)
    if cv.waitKey(10) & 0xFF == ord('q'):
                    break
cap.release()
cv.destroyAllWindows()

['helmet', 'head']
